In [1]:
pip install pandas geopandas

   ---------------------------------------- 0.0/16.2 MB ? eta -:--:--
   ------------------ --------------------- 7.6/16.2 MB 36.2 MB/s eta 0:00:01
   ---------------------------------------  16.0/16.2 MB 40.2 MB/s eta 0:00:01
   ---------------------------------------- 16.2/16.2 MB 31.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 6.3/6.3 MB 38.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 23.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df = pd.read_csv('../data/raw/pfas.csv',  encoding='ISO-8859-1')

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/pfas.csv'

In [ ]:
df.head()

In [ ]:
min_gm_result = df['gm_samp_collection_date'].min()
max_gm_result = df['gm_samp_collection_date'].max()
min_gm_result, max_gm_result

In [ ]:
# Convert the 'gm_samp_collection_date' to datetime if not already done
df['gm_samp_collection_date'] = pd.to_datetime(df['gm_samp_collection_date'])

# Filter the dataset for dates between 2017 and 2021
filtered_df = df[(df['gm_samp_collection_date'] >= '2017-01-01') & (df['gm_samp_collection_date'] <= '2021-12-31')]

df = filtered_df

# Display the filtered dataframe
df.head()

In [ ]:
df['gm_chemical_name'].value_counts()

In [ ]:
df['gm_chemical_vvl'].value_counts()

In [ ]:
df['gm_well_category'].value_counts()

In [ ]:
missing_values = df.isna().sum()
missing_values

In [ ]:
# Drop values where gm_result is NaN
df = df.dropna(subset=['gm_result'])

In [ ]:
# Group by 'gm_chemical_name' and 'year', then calculate the mean of 'gm_result'
average_result = df.groupby(['gm_chemical_name'])['gm_result'].mean().reset_index()

# Display the result
average_result

In [ ]:
california_counties = ['Alameda', 'Alpine', 'Amador', 'Butte', 'Calaveras', 'Colusa',
	'Contra Costa', 'Del Norte', 'El Dorado', 'Tulare', 'Fresno',
	'Glenn', 'Humboldt', 'Imperial', 'Inyo', 'Kern', 'Kings', 'Lake',
	'Lassen', 'Los Angeles', 'Madera', 'Marin', 'Mariposa',
	'Mendocino', 'Merced', 'Modoc', 'Mono', 'Monterey', 'Napa',
	'Nevada', 'Orange', 'Placer', 'Plumas', 'Riverside', 'Sacramento',
	'San Benito', 'San Bernardino', 'San Diego', 'San Francisco',
	'San Joaquin', 'San Luis Obispo', 'San Mateo', 'Santa Barbara',
	'Santa Clara', 'Santa Cruz', 'Shasta', 'Sierra', 'Siskiyou',
	'Solano', 'Sonoma', 'Stanislaus', 'Sutter', 'Shasta', 'Tehama',
	'Trinity', 'Tulare', 'Tuolumne', 'Ventura', 'Kern', 'Yolo', 'Yuba'
]

In [ ]:
# Extract relevant columns
df = df[['gm_chemical_vvl', 'gm_result', 'gm_samp_collection_date', 'gm_latitude', 'gm_longitude']]
df = df.sort_values(by='gm_samp_collection_date', ascending=False)
df

In [ ]:
import geopandas as gpd

# Load the shapefile
shapefile_path = '../data/raw/tl_2022_us_county.zip'
gdf = gpd.read_file(shapefile_path)

In [ ]:
gdf.columns

In [ ]:
gdf.head()

In [ ]:
# Calculate the bounds for each county
county_bounds = {}
for _, row in gdf.iterrows():
	county_name = row['NAME']  # Adjust this to match the column name in your shapefile
	bounds = row['geometry'].bounds
	county_bounds[county_name] = {
		'min_lat': bounds[1],
		'max_lat': bounds[3],
		'min_lon': bounds[0],
		'max_lon': bounds[2]
	}

In [ ]:
california_county_bounds = {name: county_bounds[name] for name in california_counties}
california_county_bounds

In [ ]:
def get_county_from_bounds(lat, lon):
	for county, bounds in california_county_bounds.items():
		if bounds['min_lat'] <= lat <= bounds['max_lat'] and bounds['min_lon'] <= lon <= bounds['max_lon']:
			return county
	return 'Unknown'

In [ ]:
df['county'] = df.apply(lambda row: get_county_from_bounds(row['gm_latitude'], row['gm_longitude']), axis=1)
df

In [ ]:
df['county'].value_counts()

In [ ]:
df = df[['gm_chemical_vvl', 'gm_result', 'gm_samp_collection_date', 'county']]
df = df.sort_values(by='gm_samp_collection_date', ascending=False)
df.head(60)

In [ ]:
grouped_df = df.groupby(['county', 'gm_samp_collection_date', 'gm_chemical_vvl'])['gm_result'].mean().reset_index()
grouped_df = grouped_df.sort_values(by='gm_samp_collection_date', ascending=False)
# Display the grouped dataframe
grouped_df.head(10)

In [ ]:
# Pivot the DataFrame to wide format
wide_df = grouped_df.pivot(index=['county', 'gm_samp_collection_date'], columns='gm_chemical_vvl', values='gm_result').reset_index()

# Display the wide format DataFrame
wide_df.head(10)

In [ ]:
# Flatten the columns and rename them
wide_df.columns.name = None
wide_df.columns = ['county', 'gm_samp_collection_date'] + [f'{col}' for col in wide_df.columns[2:]]

wide_df = wide_df.sort_values(by=['county', 'gm_samp_collection_date'], ascending=False)

# Display the wide format DataFrame with renamed columns
wide_df.head(10)

In [ ]:
wide_df.isna().sum()

In [ ]:
# Ensure the 'gm_samp_collection_date' column is in datetime format
wide_df['gm_samp_collection_date'] = pd.to_datetime(wide_df['gm_samp_collection_date'])

# Extract the year from the 'gm_samp_collection_date' column
wide_df['year'] = wide_df['gm_samp_collection_date'].dt.year

# Calculate the annual mean for each PFAS by county and year
annual_means = wide_df.groupby(['county', 'year']).mean().reset_index()
annual_means

In [ ]:
# Function to fill missing values with the annual mean
def fill_with_annual_mean(row, annual_means):
	county = row['county']
	year = row['year']
	for col in annual_means.columns[2:]:  # Skip 'county' and 'year' columns
		if pd.isna(row[col]):
			mean_value = annual_means[(annual_means['county'] == county) & (annual_means['year'] == year)][col]
			if not mean_value.empty:
					row[col] = mean_value.values[0]
	return row

# Apply the function to fill missing values
wide_df_filled = wide_df.apply(lambda row: fill_with_annual_mean(row, annual_means), axis=1)

# Drop the 'year' column as it's no longer needed
wide_df_filled = wide_df_filled.drop(columns=['year'])

wide_df_filled = wide_df_filled.sort_values(by=['county', 'gm_samp_collection_date'], ascending=False)

# Display the DataFrame with filled values
wide_df_filled.head(10)

In [ ]:
# Fill NaN values with the mean for all columns except 'county' and 'gm_samp_collection_date'
columns_to_fill = wide_df_filled.columns.difference(['county', 'gm_samp_collection_date'])
wide_df_filled[columns_to_fill] = wide_df_filled[columns_to_fill].fillna(wide_df_filled[columns_to_fill].mean())

# Display the DataFrame with filled values
wide_df_filled

In [ ]:
pfas_missing_values_by_county = wide_df_filled.isna().sum()
pfas_missing_values_by_county

In [ ]:
wide_df_filled = wide_df_filled.fillna(0)
wide_df_filled

In [ ]:
wide_df_filled = wide_df_filled.sort_values(by=['county', 'gm_samp_collection_date'])
wide_df_filled

In [ ]:
import numpy as np

# Apply log transformation to all PFAS columns (excluding county and date)
pfas_columns = wide_df_filled.columns.difference(['county', 'gm_samp_collection_date'])

# Add small constant (1) before taking log to handle zeros
wide_df_filled[pfas_columns] = np.log1p(wide_df_filled[pfas_columns])

# Display first few rows of log-transformed data
wide_df_filled.head()



In [ ]:
# Get list of PFAS chemical columns (excluding county and date)
pfas_columns = wide_df_filled.columns.difference(['county', 'gm_samp_collection_date'])

# Calculate total PFAS concentration for each row by summing across all PFAS columns
wide_df_filled['total_pfas_concentration'] = wide_df_filled[pfas_columns].sum(axis=1)

# Display the first few rows to verify the total_pfas_concentration column
print("First few rows with total PFAS concentrations:")
wide_df_filled[['county', 'gm_samp_collection_date', 'total_pfas_concentration']].head()


In [ ]:
# Save the DataFrame to a CSV file
wide_df_filled.to_csv('../data/cleaned/cleaned_pfas.csv', index=False)

In [ ]:
# Read in the cleaned cancer data
cancer_df = pd.read_csv('../data/cleaned/cleaned_cancer_dataset.csv')

# Merge PFAS and cancer data on county
merged_df = pd.merge(wide_df_filled, cancer_df, on='county', how='inner')

# Display first few rows of merged dataset
print("First few rows of merged PFAS and cancer data:")
merged_df


In [ ]:
# Save the merged DataFrame to a CSV file
merged_df.to_csv('../data/cleaned/cleaned_pfas_cancer_merged.csv', index=False)
